# StarDist for counting Cell Nuclei From Fluorescence Imaging
## Introduction

This script is meant to:

1. Iterate all files in a provided directory,
2. First perform cell counting on DAPI images and output a file `parameter-timepoint-DAPI.out`
3. Then perform cell counting on FITC images and output a fle `parameter-timepoint-FITC.out`
4. Add the FITC output to the DAPI output, compute the fractional cell viability, then save as `viability-parameter.csv`


## Parameters Needed

**This notebook assumes that each experimental group is isolated by directory.**

**If isolated by filename, please make sure file names are semantic.** 

+ Directory of images to be analyzed **Ensure that each image files has _DAPI.tif or _FITC.tif as a suffix**
+ Parameter name, i.e. what is being examined
+ If this consists of multiple time points, the time point


## Step One: Initialize Requisite Libraries

This makes sure that libraries required for StarDist are loaded. Tensorflow may ask to rebuild for AVX instructions, this is not necessary.

In [2]:
from stardist.models import StarDist2D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt
from skimage.segmentation import find_boundaries
from tifffile import imread
from skimage.color import label2rgb
import os
from glob import glob
import pandas as pd  

data = [] # initialize empty data frame

2025-01-26 20:26:22.997523: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Step Two: Load StarDist 2D Fluorescence Training Set

StarDist is trained on the following datasets

+ 2D nuclear fluoresence
+ 2D Hematoxylin and Eosin (H&E) histological sections
+ 3D nuclear fluoresence

You need to initialize the training data you need for proper function, as shown here

In [9]:
model = StarDist2D.from_pretrained('2D_versatile_fluo') # 2D Nuclei Fluorescence

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


2025-01-26 20:26:59.718382: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Step Three: Specify Your Parameters

First, specify the directory you have your image files at, an example is 

`imageDirectory = os.path.expanduser("/home/woodn/Desktop/viability/images/")`

In [14]:
imageDirectory = os.path.expanduser("<directory/here>")
saveDirectory = os.path.expanduser(imageDirectory)

Now enter what this image set represents

An example would be:

`experimentalGroup1`

Replace white space with characters such as -,_,., or use camelCaseLikeThis

In [15]:
setName = '<enter_set_name_here>'

If needed, designate a time point, otherwise, keep it as `na`

In [8]:
timePoint = 'na'

## Step Four: Run StarDist on DAPI Files

The below command will read all DAPI files. 

Please make sure that 

+ You specify a new directory for each experimental group, otherwise, you are executing for the same group again and again
+ Specify experimental group name
+ Specify time point if used

In [13]:
dapiout = f'{setName}-{timePoint}-DAPI.out'
saveReadsCSV = os.path.join(saveDirectory,dapiout)
dapi = []
for img in sorted(glob(os.path.join(imageDirectory,"*_DAPI.tif"))):
    image = imread(img)
    image = normalize(image,1,99.8)         # NORMALIZE FLUOR INTENSITY
    labels, cells = model.predict_instances(image,
        scale=1,
        return_labels=True)             # THIS IS THE MODEL 
    probability = list(cells["prob"])
    n_detections = len(probability)     # THE LENGTH OF PROBABILITY VECTOR IS THE CELL COUNT
    print('\n')
    print(f'{img},{setName},{timePoint},{n_detections}')     # PRINTS OUT EACH ITERATED FILE
    dapi.append([timePoint,setName,img, n_detections])    # STORE CELL COUNT WITH FILE NAME



/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph3-day01.tif,<enter_set_name_here>,na,1385


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph3-day02.tif,<enter_set_name_here>,na,686


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph3-day03.tif,<enter_set_name_here>,na,1381


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph3-day04.tif,<enter_set_name_here>,na,2204


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph3-day05.tif,<enter_set_name_here>,na,1729


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph5-day01.tif,<enter_set_name_here>,na,1390


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph5-day02.tif,<enter_set_name_here>,na,1611


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph5-day03.tif,<enter_set_name_here>,na,2187


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph5-day04.tif,<enter_set_name_here>,na,2422


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph5-day05.tif,<enter_set_name_here>,na,2714


/home/woodn/Desktop/viabilityph/DAPI/01_DAPI-ph6-day01.tif,<enter_set_name_here>,na,1523


Now save your DAPI data

In [ ]:
dataFrame = pd.DataFrame(dapi, columns = ["timePoint","setName","FILENAME","COUNT"])
dataFrame.to_csv(saveReadsCSV, sep='\t', encoding='utf-8') 

## Step Five: Run StarDist on FITC Files

This performs the same function as StarDist on DAPI, but now on FITC channel images

In [ ]:
fitcout = f'{setName}-{timePoint}-FITC.out'
saveReadsCSV = os.path.join(saveDirectory,fitcout)
fitc = []
for img in sorted(glob(os.path.join(imageDirectory,"*_FITC.tif"))):
    image = imread(img)
    image = normalize(image,1,99.8)         # NORMALIZE FLUOR INTENSITY
    labels, cells = model.predict_instances(image,
        scale=1,
        return_labels=True)             # THIS IS THE MODEL 
    probability = list(cells["prob"])
    n_detections = len(probability)     # THE LENGTH OF PROBABILITY VECTOR IS THE CELL COUNT
    print('\n')
    print(f'{img},{setName},{timePoint},{n_detections}')     # PRINTS OUT EACH ITERATED FILE
    fitc.append([timePoint,setName,img, n_detections])    # STORE CELL COUNT WITH FILE NAME

Now save your FITC data !

In [ ]:
dataFrame = pd.DataFrame(fitc, columns = ["timePoint","setName","FILENAME","COUNT"])
dataFrame.to_csv(saveReadsCSV, sep='\t', encoding='utf-8') 

## Outcome

You should have

+ 2 CSV files, one for each fluoresence channel, in the specified output directory, which by default is the input directory